In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy


warnings.filterwarnings("ignore")

from riverlakenetwork import DataChecker, DataLoader
from riverlakenetwork import ResolvableLakes
from riverlakenetwork import BurnLakes

In [2]:
# Load general configurations from a YAML file
yaml_file = os.path.abspath('../code/HydroLakes_MERITBasins/HydroLakes_MERITBasins_Config.yaml')
with open(yaml_file, 'r') as f:
    GeneralConfig = yaml.safe_load(f)

# get the path to the merit-basins files
path_out             = GeneralConfig['DomainFolder']
TempFolder           = GeneralConfig['TempFolder']
pfafs                = GeneralConfig['pfafs']
lake_file            = GeneralConfig['LakeFile']
IncludeLakes         = GeneralConfig['IncludeLakes']
riv_path             = GeneralConfig['MERITBasinsPaths']['RivPath']
cat_path             = GeneralConfig['MERITBasinsPaths']['CatPath']
cst_path             = GeneralConfig['MERITBasinsPaths']['CstPath']
riv_file_template    = GeneralConfig['MERITBasinsPaths']['RivFileTemplate']
cat_file_template    = GeneralConfig['MERITBasinsPaths']['CatFileTemplate']
cst_file_template    = GeneralConfig['MERITBasinsPaths']['CstFileTemplate']

In [3]:
if IncludeLakes:
    #load hydrolakeDataset
    lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
    lakeO = lakeO.sample(frac=0.10, random_state=42)
    #lakeO = FixHydroLakesForMerit(lakeO) # correct for hydrolakes for merit
    lakeO['x'] = lakeO.centroid.x # add centroid lon
    lakeO['y'] = lakeO.centroid.y # add centroid lat

# loop over the config files:
for pfaf in ['71']:

    # get the list of pfaf for the subdomain
    subdomain = 'pfaf'+pfaf
    
    # create the folder and domain to include
    path_out_temp = TempFolder+'pfaffff'+pfaf+'/'+subdomain+'/'
    if os.path.isdir(path_out_temp):
        try:
            shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
        except OSError as e:
            print(f"Error: {path_out_temp} - {e}")
    if not os.path.isdir(path_out_temp):
        os.makedirs(path_out_temp)

    # read in the files
    riv, cat = gm.merit_read_file ([pfaf],
                                   riv_path,
                                   riv_file_template,
                                   cat_path,
                                   cat_file_template,
                                   path_cst = cst_path,
                                   cst_file_template = cst_file_template)
    
    # prepare the ntopo for merit
    riv, cat = gm.prepare_ntopo(riv=riv,\
                                riv_cols={'id':'COMID', 'next_id':'NextDownID'},\
                                cat=cat,\
                                cat_cols={'id':'COMID', 'hillslope':'hillslope', 'area' :'unitarea'},\
                                network = 'merit')

In [4]:
import riverlakenetwork as rlt

# use DataLoader to
# Example config dictionary
config = {
    "riv": riv,
    "riv_dict": {
        "COMID": {"col":"COMID"},
        "NextDownCOMID": {"col":"NextDownID"},
        "length": {"col":"lengthkm"},
        "uparea": {"col":"uparea","unit":"km2"}
    },

    "cat": cat,
    "cat_dict": {
        "COMID": {"col":"COMID"},
        "unitarea": {"col":"unitarea","unit":"km2"},
    },

    "lake": lakeO,
    "lake_dict": {
        "LakeCOMID": {"col":"Hylak_id"},
        "unitarea": {"col":"Lake_area","unit":"km2"}
    }
}

# Create DataLoader instance
RLT = rlt.DataLoader(config)

# Access the layers
riv = RLT.riv
cat = RLT.cat
lake = RLT.lake


riv: Loaded
riv_dict: {'COMID': {'col': 'COMID'}, 'NextDownCOMID': {'col': 'NextDownID'}, 'length': {'col': 'lengthkm'}, 'uparea': {'col': 'uparea', 'unit': 'km2'}}
cat: Loaded
cat_dict: {'COMID': {'col': 'COMID'}, 'unitarea': {'col': 'unitarea', 'unit': 'km2'}}
lake: Loaded
lake_dict: {'LakeCOMID': {'col': 'Hylak_id'}, 'unitarea': {'col': 'Lake_area', 'unit': 'km2'}}


In [5]:
lake


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry,x,y
664405,664406,None,Canada,North America,CanVec,1,0,0.17,2.19,1.50,...,0.006,683.9,420,0.89,0.8,-101.994422,59.574059,"POLYGON ((-101.99237 59.57637, -101.99253 59.5...",-101.996010,59.574377
346374,346375,None,Canada,North America,CanVec,1,0,0.32,3.50,1.74,...,0.007,3293.6,338,6.88,1.1,-113.402532,67.272917,"POLYGON ((-113.4012 67.27549, -113.40115 67.27...",-113.403898,67.270445
1299627,1299628,None,United Kingdom,Europe,SWBD,1,0,0.13,1.90,1.50,...,0.068,57.3,40,1.60,3.5,-3.125567,58.368750,"POLYGON ((-3.12625 58.365, -3.12819 58.36502, ...",-3.125364,58.368798
466269,466270,None,Canada,North America,CanVec,1,0,0.12,1.87,1.52,...,0.002,1850.2,297,1.61,0.3,-104.801042,64.689512,"POLYGON ((-104.80481 64.69101, -104.79988 64.6...",-104.800933,64.689437
161736,161737,None,Russia,Europe,SWBD,1,0,2.96,6.34,1.04,...,0.007,6202.6,130,0.44,20.3,69.738542,55.546096,"POLYGON ((69.74873 55.53002, 69.74861 55.52986...",69.743179,55.537684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863928,863929,None,Canada,North America,CanVec,1,0,0.77,13.08,4.22,...,0.886,26.6,353,0.92,41.5,-73.981250,53.491193,"POLYGON ((-73.97932 53.50309, -73.97786 53.503...",-73.987201,53.496443
437165,437166,None,Canada,North America,CanVec,1,0,0.65,3.55,1.25,...,0.015,1534.3,155,1.31,1.1,-72.331250,65.303488,"POLYGON ((-72.32425 65.30876, -72.31722 65.306...",-72.326727,65.305693
1006147,1006148,None,Canada,North America,CanVec,1,0,0.12,1.64,1.34,...,0.046,101.1,352,2.70,1.7,-84.703125,47.750796,"POLYGON ((-84.70485 47.75236, -84.70431 47.752...",-84.703055,47.750138
716107,716108,None,Canada,North America,CanVec,1,0,0.34,2.26,1.10,...,0.019,268.6,51,0.28,1.8,-93.770254,58.049242,"POLYGON ((-93.76932 58.05363, -93.76768 58.052...",-93.773240,58.051267


In [6]:
checker = DataChecker(loaded_data=RLT)
checker

Subbasin and lake area units are consistent: km2
riv CRS: EPSG:4326
cat CRS: EPSG:4326
lake CRS: EPSG:4326


In [7]:
checker.riv

,COMID,NextDownCOMID,length,uparea,geometry
0,71000001,71000065,7.020492,391.901537,"LINESTRING (-98.06167 60, -98.0625 59.99917, -..."
1,71000002,71000065,24.171249,701.754912,"LINESTRING (-98.06167 60, -98.06083 60.00083, ..."
2,71000003,71000002,14.760464,480.760660,"LINESTRING (-98.1725 60.13, -98.17333 60.12917..."
3,71000004,71000003,6.240078,388.938962,"LINESTRING (-98.335 60.07, -98.33583 60.07083,..."
4,71000005,71000004,4.462477,328.557527,"LINESTRING (-98.39917 60.09667, -98.4 60.09583..."
...,...,...,...,...,...
50682,71050683,-9999,0.001000,0.060656,None
50683,71050684,-9999,0.001000,0.008723,None
50684,71050685,-9999,0.001000,0.004362,None
50685,71050686,-9999,0.001000,10.810153,None


In [8]:
checker.cat

,COMID,unitarea,geometry
0,71000001,39.620826,"POLYGON ((-98.14542 60.06375, -98.13792 60.063..."
1,71000002,99.276774,"MULTIPOLYGON (((-98.11958 60.19292, -98.11458 ..."
2,71000003,63.957629,"MULTIPOLYGON (((-98.33375 60.03542, -98.33292 ..."
3,71000004,16.746171,"MULTIPOLYGON (((-98.40125 60.05208, -98.40042 ..."
4,71000005,5.993164,"POLYGON ((-98.43208 60.10708, -98.43125 60.107..."
...,...,...,...
50682,71050683,0.060656,"POLYGON ((-94.80792 59.88292, -94.80458 59.882..."
50683,71050684,0.008723,"POLYGON ((-94.80875 59.65792, -94.80708 59.657..."
50684,71050685,0.004362,"POLYGON ((-94.81042 59.65792, -94.80958 59.657..."
50685,71050686,10.810153,"POLYGON ((-108.54708 52.72625, -108.54625 52.7..."


In [9]:
checker.lake

,LakeCOMID,unitarea,geometry
664405,664406,0.17,"POLYGON ((-101.99237 59.57637, -101.99253 59.5..."
346374,346375,0.32,"POLYGON ((-113.4012 67.27549, -113.40115 67.27..."
1299627,1299628,0.13,"POLYGON ((-3.12625 58.365, -3.12819 58.36502, ..."
466269,466270,0.12,"POLYGON ((-104.80481 64.69101, -104.79988 64.6..."
161736,161737,2.96,"POLYGON ((69.74873 55.53002, 69.74861 55.52986..."
...,...,...,...
863928,863929,0.77,"POLYGON ((-73.97932 53.50309, -73.97786 53.503..."
437165,437166,0.65,"POLYGON ((-72.32425 65.30876, -72.31722 65.306..."
1006147,1006148,0.12,"POLYGON ((-84.70485 47.75236, -84.70431 47.752..."
716107,716108,0.34,"POLYGON ((-93.76932 58.05363, -93.76768 58.052..."


In [10]:
resolvable = ResolvableLakes(cat=checker.cat, lake=checker.lake, riv=checker.riv, margin=2.0)



Index(['LakeCOMID', 'unitarea_left', 'geometry', 'x', 'y', 'index_right',
       'COMID', 'unitarea_right'],
      dtype='object')
==== Number of lakes after subsetting: 14101 ====
==== Number of lakes after removing intersection with only one lake: 521 ====
==== Number of lakes after removing lakes that do not touch starting or ending points of river segments: 416 ====
==== Number of lakes after removing lakes that do touch only one starting or ending points of river segments: 407 ====
==== Number of lakes after removing lakes that do intersect with only one river segment: 407 ====
==== Number of lakes after removing lakes that do have exactly the same uparea for their maximume uparea for various: 407 ====
==== Number of lakes after removing lakes from segments that intersect with more than 3 lakes: 407 ====
==== Number of lakes after enforcing one lake per river segment: 383 ====
==== Number of lakes after identifying the graph number within a lake: 383 ====


In [11]:
resolvable.lake_resolvable.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/test.gpkg")

In [12]:
corrected = BurnLakes(cat=checker.cat, lake=resolvable.lake_resolvable, riv=checker.riv)

corrected.cat_corrected.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/cat1.gpkg")
corrected.riv_corrected.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/riv1.gpkg")

In [13]:
corrected.riv_corrected

,COMID,NextDownCOMID,length,uparea,geometry,LakeCOMID,unitarea,exorheic,endorheic,islake,inflow,outflow,inoutflow
0,71000001,71000065.0,7.020492,391.901537,"LINESTRING (-98.06167 60, -98.0625 59.99917, -...",NaN,39.620826,NaN,NaN,NaN,0,0,0
1,71000002,71000065.0,24.171249,220.994252,"LINESTRING (-98.06167 60, -98.06083 60.00083, ...",NaN,99.276774,NaN,NaN,NaN,0,0,0
2,71000003,71050902.0,9.937119,61.743318,"MULTILINESTRING ((-98.1725 60.13, -98.17333 60...",NaN,61.743318,NaN,NaN,NaN,0,1,0
3,71000004,71050902.0,5.729200,388.457646,"LINESTRING (-98.34031 60.07281, -98.34083 60.0...",NaN,16.264855,NaN,NaN,NaN,1,0,0
4,71000005,71000004.0,4.462477,328.557527,"LINESTRING (-98.39917 60.09667, -98.4 60.09583...",NaN,5.993164,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51065,71051066,71033619.0,NaN,93585.694241,"POLYGON ((-95.70553 50.18981, -95.70711 50.189...",7916.0,21.480000,1.0,0.0,1.0,0,0,0
51066,71051067,71007519.0,NaN,113123.475222,"POLYGON ((-104.36507 55.59438, -104.36486 55.5...",512.0,117.370000,1.0,0.0,1.0,0,0,0
51067,71051068,71004359.0,NaN,65332.590635,"POLYGON ((-100.07315 56.57445, -100.07625 56.5...",471.0,419.540000,1.0,0.0,1.0,0,0,0
51068,71051069,71011349.0,NaN,606553.035910,"POLYGON ((-98.02161 54.53363, -98.01986 54.533...",553.0,358.570000,1.0,0.0,1.0,0,0,0


In [14]:
print(corrected.riv_corrected[corrected.riv_corrected["COMID"] == 71038983])


          COMID  NextDownCOMID  length    uparea geometry  LakeCOMID  \
38982  71038983     71051065.0     0.0  56.21229     None        NaN   

       unitarea  exorheic  endorheic  islake  inflow  outflow  inoutflow  
38982  56.21229       NaN        NaN     NaN       1        0          0  
